# NASA Turbofan Engine Dataset Visualization 3

##### 여기서부터는 엔진별, 센서별로 분할된 데이터를 센서별로 1번부터 100번 엔진까지의 데이터를 한 차트에 보여줄 수 있도록 병합하는 코드를 만들겠다. 

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from pathlib import PurePath
from pandas import DataFrame
import xlwings as xw
import xlsxwriter

### NASA Turbofan Engine Dataset Visualization 2에서 지정한 변수들 불러오기
    
#### FD001,2,3,4 Train 데이터셋 구조 파악 (FD001,2,3,4 Test 데이터 불러오기)

### 1번 ~ 100번 엔진까지, 각 센서별 데이터로 재추출하는 코드

이전에는 엔진별로 서로 다른 종류의 모든 센서 데이터를 하나의 시계열 그래프로 나타내었다면,
이번에는 엔진번호(Unit_Number), 비행시간(Time), 센서 1개에서 나온 데이터만 추출해보겠다.
그리고 나중에는 이를 바탕으로 특정 한 센서를 중심으로 각 엔진들을 join한 형태의 테이블을 작성해보겠다.

그전에 우선 FD001의 train에서 4번엔진의 비행 싸이클인 'Time','Fan Inlet Temp' 데이터만 추출해보겠다.
필요한 코드는 아래와 같다.

#### FD001 train set 센서별 데이터 병합
#####
##### FD001 train set '01. Fan Inlet Temp' 데이터 병합

In [9]:
# 01. Fan Inlet Temp 데이터 병합


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 01. Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 센서별 & 엔진별 분할된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 & 엔진별 분할 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()               # 센서별 데이터 병합 종료

print('생성 파일 : ', excel_file)    # 생성한 파일 이름 출력

생성 파일 :  C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/FD001 train 01. Fan Inlet Temp.xlsx


##### FD001 train set '01. Fan Inlet Temp' Time serise chart (시계열 그래프)

In [3]:
# 01. Fan Inlet Temp 데이터  Time serise chart (시계열 그래프)

# 데이터 불러오기

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 01. Fan Inlet Temp.xlsx'

Joined = pd.read_excel(excel_file)




# (1) 판다스 Excel Writer 객체 생성

folder1 = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

file = 'FD001 train 01. Fan Inlet Temp '

TSC = 'Time Seirse Chart.xlsx'

name = folder1 + file + TSC

excel_writer = pd.ExcelWriter(name, engine = 'xlsxwriter')




# (2) DataFrame 데이터를 지정된 워크시트(sheet_name=시트_이름)에 쓰기

worksheet_name = 'Sheet1' # 워크시트 이름을 변수에 할당

Joined.to_excel(excel_writer, sheet_name = worksheet_name, index = False)




# (3) ExcelWriter 객체에서 워크북(workbook)과 워크시트(worksheet) 객체생성

workbook = excel_writer.book

worksheet = excel_writer.sheets[worksheet_name]




# (4) 차트 객체 생성(원하는 차트의 종류 지정)

chart = workbook.add_chart({'type':'line'}) # 꺾은선형 차트




# (5) 차트를 생성하기 위한 데이터 값의 범위 지정

columns_len = len(Joined.columns) # df.shape[1]도 가능



for k in range(columns_len -1):
    
    start_row = 1                      # 범위의 시작 행 번호

    start_column = k + 1                # 범위의 시작 열 번호

    end_row = len(Joined.index)            # 범위의 끝 행 번호 

    end_column = k + 1                 # 범위의 끝 열 번호



    # 그래프 및 축 값을 위한 데이터 범위, 범례 이름, 마커 지정

    chart.add_series({

        'values':[worksheet_name, start_row, start_column, end_row, end_column], # 꺾은선형 차트를 위한 데이터 범위 지정

        'categories':[worksheet_name, start_row, 0, end_row, 0],                 # x 축 값에 사용할 데이터 범위 지정

        'name': [worksheet_name, 0, k + 1]                                      # 범례 이름에 사용할 데이터 지정

        })



# (5-1) 엑셀 차트에 차트 제목과 x,y 축 제목 추가

chart.set_title({'name': 'FD001 train 01. Fan Inlet Temp Time Seirse Chart'})   # 차트 제목 지정

chart.set_x_axis({'name':'Cycle'})                                              # x축 제목 지정

chart.set_y_axis({'name': '단위 : °R (Rankine)'})                                # y축 제목은 미지정



# (6) 워크시트에 차트가 들어갈 위치 지정해 차트 넣기

worksheet.insert_chart(1, columns_len, chart, {'x_offset' : 100, 'y_offset' : 90})



# (7) ExcelWriter 객체를 닫고 엑셀 파일을 출력

excel_writer.save()

print('생성 파일:', name)                                # 생성한 파일 이름 출력


생성 파일: C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/FD001 train 01. Fan Inlet Temp Time Seirse Chart.xlsx


해당 엑셀파일을 열어보면 아래와 같다.

In [14]:
example = pd.read_excel('C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/FD001 train 01. Fan Inlet Temp.xlsx')

example

,Time (Flight Cycle),1번 엔진,2번 엔진,3번 엔진,4번 엔진,5번 엔진,6번 엔진,7번 엔진,8번 엔진,9번 엔진,...,91번 엔진,92번 엔진,93번 엔진,94번 엔진,95번 엔진,96번 엔진,97번 엔진,98번 엔진,99번 엔진,100번 엔진
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396,397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
397,398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
398,399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 02. LPC Outlet Temp
# 03. HPC Outlet Temp
# 04. LPT Outlet Temp
# 05. Fan Inlet Press
# 06. Bypass Duct Press
# 07. Total HPC Outlet Press
# 08. Physical Fan Speed
# 09. Physical Core Speed
# 10. Engine Press Ratio
# 11. Static HPC Outlet Press
# 12. Fuel Flow Ratio to Ps30
# 13. Corrected Fan Speed
# 14. Corrected Corr Speed
# 15. Bypass Ratio
# 16. Burner Fuel-Air ratio
# 17. Bleed Enthalpy
# 18. Demanded Fan Speed
# 19. Demanded Corrected Fan Speed
# 20. HPT Collant Bleed
# 21. LPT_Coolant_Bleed

In [15]:
# 02. LPC Outlet Temp



# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 02. LPC Outlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 센서별 & 엔진별 분할된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 & 엔진별 분할 데이터 모음/train/FD001/02. LPC Outlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 02. LPC Outlet Temp 파일 병합이 완료되었습니다.')

FD001 train 02. LPC Outlet Temp 파일 병합이 완료되었습니다.


In [16]:
# 03. HPC Outlet Temp



# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 03. HPC Outlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 센서별 & 엔진별 분할된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 & 엔진별 분할 데이터 모음/train/FD001/03. HPC Outlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 03. HPC Outlet Temp 파일 병합이 완료되었습니다.')

FD001 train 03. HPC Outlet Temp 파일 병합이 완료되었습니다.


In [17]:
# 04. LPT Outlet Temp



# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 04. LPT Outlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 센서별 & 엔진별 분할된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 & 엔진별 분할 데이터 모음/train/FD001/04. LPT Outlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 04. LPT Outlet Temp 파일 병합이 완료되었습니다.')

FD001 train 04. LPT Outlet Temp 파일 병합이 완료되었습니다.


In [18]:
# 05. Fan Inlet Press



# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 05. Fan Inlet Press.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 센서별 & 엔진별 분할된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 & 엔진별 분할 데이터 모음/train/FD001/05. Fan Inlet Press/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 05. Fan Inlet Press 파일 병합이 완료되었습니다.')

FD001 train 05. Fan Inlet Press 파일 병합이 완료되었습니다.


In [19]:
# 06. Bypass Duct Press



# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 06. Bypass Duct Press.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 센서별 & 엔진별 분할된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 & 엔진별 분할 데이터 모음/train/FD001/06. Bypass Duct Press/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 06. Bypass Duct Press 파일 병합이 완료되었습니다.')

FD001 train 06. Bypass Duct Press 파일 병합이 완료되었습니다.


In [20]:
# 07. Total HPC Outlet Press



# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 07. Total HPC Outlet Press.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 센서별 & 엔진별 분할된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 & 엔진별 분할 데이터 모음/train/FD001/07. Total HPC Outlet Press/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 07. Total HPC Outlet Press 파일 병합이 완료되었습니다.')

FD001 train 07. Total HPC Outlet Press 파일 병합이 완료되었습니다.


In [21]:
# 08. Physical Fan Speed



# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 08. Physical Fan Speed.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 센서별 & 엔진별 분할된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 & 엔진별 분할 데이터 모음/train/FD001/08. Physical Fan Speed/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 08. Physical Fan Speed 파일 병합이 완료되었습니다.')

FD001 train 08. Physical Fan Speed 파일 병합이 완료되었습니다.


In [22]:
# 09. Physical Core Speed



# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 09. Physical Core Speed.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 센서별 & 엔진별 분할된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 & 엔진별 분할 데이터 모음/train/FD001/09. Physical Core Speed/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 09. Physical Core Speed 파일 병합이 완료되었습니다.')

FD001 train 09. Physical Core Speed 파일 병합이 완료되었습니다.


In [23]:
# 10. Engine Press Ratio



# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 10. Engine Press Ratio.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 센서별 & 엔진별 분할된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 & 엔진별 분할 데이터 모음/train/FD001/10. Engine Press Ratio/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 10. Engine Press Ratio 파일 병합이 완료되었습니다.')

FD001 train 10. Engine Press Ratio 파일 병합이 완료되었습니다.


In [24]:
# 11. Static HPC Outlet Press



# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 11. Static HPC Outlet Press.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 센서별 & 엔진별 분할된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 & 엔진별 분할 데이터 모음/train/FD001/11. Static HPC Outlet Press/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 11. Static HPC Outlet Press 파일 병합이 완료되었습니다.')

FD001 train 11. Static HPC Outlet Press 파일 병합이 완료되었습니다.


In [25]:
# 12. Fuel Flow Ratio to Ps30



# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 12. Fuel Flow Ratio to Ps30.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 센서별 & 엔진별 분할된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 & 엔진별 분할 데이터 모음/train/FD001/12. Fuel Flow Ratio to Ps30/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 12. Fuel Flow Ratio to Ps30 파일 병합이 완료되었습니다.')

FD001 train 12. Fuel Flow Ratio to Ps30 파일 병합이 완료되었습니다.


In [26]:
# 13. Corrected Fan Speed



# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 13. Corrected Fan Speed.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 센서별 & 엔진별 분할된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 & 엔진별 분할 데이터 모음/train/FD001/13. Corrected Fan Speed/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 13. Corrected Fan Speed 파일 병합이 완료되었습니다.')

FD001 train 13. Corrected Fan Speed 파일 병합이 완료되었습니다.


In [27]:
# 14. Corrected Corr Speed



# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 14. Corrected Corr Speed.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 센서별 & 엔진별 분할된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 & 엔진별 분할 데이터 모음/train/FD001/14. Corrected Corr Speed/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 14. Corrected Corr Speed 파일 병합이 완료되었습니다.')

FD001 train 14. Corrected Corr Speed 파일 병합이 완료되었습니다.


In [ ]:
# 14. Corrected Corr Speed



# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 병합 데이터 모음/train/FD001/'

excel_file = folder + 'FD001 train 14. Corrected Corr Speed.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 센서별 & 엔진별 분할된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 & 엔진별 분할 데이터 모음/train/FD001/14. Corrected Corr Speed/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 14. Corrected Corr Speed 파일 병합이 완료되었습니다.')